In [ ]:
import pandas as pd
from cortex import Cortex
import pydash
from cortex_common.types import EntityEvent, ListAttributeValue, DimensionalAttributeValue, StringAttributeValue
cortex= Cortex.client()
builder = cortex.builder()



In [ ]:
df = pd.read_csv("../../data/sample-member-profile.csv", dtype={'member_adress_zip':'str', 'profile_id' : 'str'})

In [ ]:
df = df.astype('object')

In [ ]:
list_members = df['member_id'][0:5]

In [ ]:
df2 = df[df["member_id"] .isin(list_members) ]

In [ ]:
df.head()

In [ ]:
list_columns = df.columns.tolist()

In [ ]:
cortex_schema_version = 'accelerators/Member'
cortex_schema_version

In [ ]:
event_list = [
    [EntityEvent(event = i,entityId = df['member_id'][idx],entityType = cortex_schema_version,properties = {'value': df[i][idx]}) for i in list_columns]
    for idx in df.index]

In [ ]:
event_list[0:10]

In [ ]:
for event in event_list:
    try:
        profile = builder.profiles(cortex_schema_version).with_events(event).build()
        print(profile)
    except Exception as e:
        print('Error:', e)


In [ ]:
df1 = pd.read_csv('../../data/sample-alert-data.csv', dtype={ 'member_id' : 'str'})

In [ ]:
df2 = pd.read_csv('../../data/sample-alert-acted.csv', dtype={ 'member_id' : 'str'})

In [ ]:
df1.head()

In [ ]:
df1 = df1.astype('object')

In [ ]:
member_id_list1 = []
event_list1 = []

for idx in df1.index:
    if df1.iloc[idx]['member_id'] in member_id_list1:
        break
    a = df1[df1['member_id'] == df1.iloc[idx]['member_id']]
    list_alerts = a.to_dict('records')
    val_strings = []
    for i in list_alerts:
        i.pop('member_id')
        i.pop('alert')
        val = StringAttributeValue(value=i['alert_id'])
        val_strings.append(val)
  
    list_alerts = ListAttributeValue(value=val_strings)
    event = EntityEvent(
            event = 'alerts_generated',
            entityId = df1['member_id'][idx],
            entityType = cortex_schema_version,
            properties = dict(list_alerts))
    event_list1.append(event)
    member_id_list1.append(df1.iloc[idx]['member_id'])
print(event_list1)

try:
    profile = builder.profiles(cortex_schema_version).with_events(event_list1).build()
    print(profile)
except Exception as e:
    print('Error:', e)


In [ ]:
df2 = df2.astype('object')

In [ ]:
df2.head()

In [ ]:
from cortex_common.types.attribute_values import Dimension, StringAttributeValue, CounterAttributeValue
member_id_list2 = []
event_list2 = []
for idx in df2.index:
    if df2.iloc[idx]['member_id'] in member_id_list2:
        break
    a = df2[df2['member_id'] == df2.iloc[idx]['member_id']]
    list_alerts = a.to_dict('records')
    list_alerts_viewed = [pydash.merge(pydash.pick(alert, 'alert_id'), pydash.pick(alert,'alert_viewed')) for alert in list_alerts]
    list_alerts_acted = [pydash.merge(pydash.pick(alert, 'alert_id'), pydash.pick(alert,'alert_acted')) for alert in list_alerts]
    
    viewed = [ a["alert_id"] for a in list_alerts_viewed if a["alert_viewed"] == 1]
    acted = [ a["alert_id"] for a in list_alerts_acted if a["alert_acted"] == 1]
    
    #      
    attribute_for_alerts_viewed = DimensionalAttributeValue(value=[
        Dimension(
            dimensionId=alert["alert_id"],
            dimensionValue=CounterAttributeValue(value=1 if alert["alert_id"] in viewed else 0),
        )
        for alert in list_alerts
    ], contextOfDimension=StringAttributeValue.detailed_schema_type().outerType,contextOfDimensionValue=CounterAttributeValue.detailed_schema_type())
    
    attribute_for_alerts_acted = DimensionalAttributeValue(value=[
        Dimension(
            dimensionId=alert["alert_id"],
            dimensionValue=CounterAttributeValue(value=1 if alert["alert_id"] in acted else 0),
        )
        for alert in list_alerts
    ], contextOfDimension=StringAttributeValue.detailed_schema_type().outerType,contextOfDimensionValue=CounterAttributeValue.detailed_schema_type().outerType)

#     print(attribute_for_alerts_veiwed)

    #     print(list_alerts)
#     list_alerts_viewed = ListAttributeValue(value=list_alerts_viewed)
#     list_alerts_acted = ListAttributeValue(value=list_alerts_acted)
    event_alert_viewed = EntityEvent(
            event = 'alerts_viewed',
            entityId = df2['member_id'][idx],
            entityType = cortex_schema_version,
            properties = dict(attribute_for_alerts_viewed))
    event_alert_acted = EntityEvent(
            event = 'alerts_acted',
            entityId = df2['member_id'][idx],
            entityType = cortex_schema_version,
            properties = dict(attribute_for_alerts_acted))
    event_list2.append(event_alert_viewed)
    event_list2.append(event_alert_acted)
    member_id_list2.append(df2.iloc[idx]['member_id'])
# print(event_list2)   
print(event_list2)
try:
    profile = builder.profile(cortex_schema_version).with_events(event_list2).build()
except Exception as e:
    print('Error:', e)

In [ ]:
if ':' not in cortex_schema_version:
    
    schema_version = cortex.profile_schema(cortex_schema_version).latest()._version

    cortex_schema_version = "{}:{}".format(cortex_schema_version,schema_version)


profile_attrs = cortex.profile('1234567').latest("accelerators/Member")

In [ ]:
profile_attrs